In [1]:
import psycopg2 as p
import glob, os, csv, time
from datetime import datetime


In [2]:
password = '*****'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

# The analyst changes will need to be reflected in the fs_* fields in order for the changes to be picked up (fs_tractors_owned, fs_tractors_leased, fs_trailers_owned, etc.)

## Updating Fleet totals

In [3]:
# Updates the fleet and fleet_primary table from 
cur.execute("""
with updated as (
    update fleetseek.fleet
    set 
        trucks_owned = sub_q.trucks_owned,
        trucks_leased = sub_q.trucks_leased,
        tractors_owned = sub_q.tractors_owned,
        tractors_leased = sub_q.tractors_leased,
        trailers_owned = sub_q.trailers_owned,
        trailers_leased = sub_q.trailers_leased,
        haz_trailers_owned = sub_q.haz_trailers_owned,
        haz_trailers_leased = sub_q.haz_trailers_leased,
        haz_trucks_owned = sub_q.haz_trucks_owned,
        haz_trucks_leased = sub_q.haz_trucks_leased,
        total_trucks = sub_q.total_trucks,
        total_tractors = sub_q.total_tractors,
        total_trailers = sub_q.total_trailers,
        total_vehicles = sub_q.total_vehicles,
        total_vehicles_owned = sub_q.total_vehicles_owned,
        total_vehicles_leased = sub_q.total_vehicles_leased,
        dot_hmind = sub_q.hm_ind,
        year_mileage = sub_q.mcs150mileageyear,
        vehicle_miles = sub_q.mlg150,
        drivers = sub_q.tot_drs,
        cdl_drivers = sub_q.cdl_drivers,
        update_date = now()::date,
        mcn = sub_q.mcn,
        gvw_class_12 = sub_q.gvw_class_12,
        gvw_class_345 = sub_q.gvw_class_345,
        gvw_class_6 = sub_q.gvw_class_6,
        gvw_class_7 = sub_q.gvw_class_7,
        gvw_class_8 = sub_q.gvw_class_8,
        website = sub_q.website,
        sic_code = sub_q.sic,
        product_code = sub_q.product_code
        
        

    from (
        select 
            fs_trucks_owned as trucks_owned, 
            fs_trucks_leased as trucks_leased,
            fs_tractors_owned as tractors_owned,
            fs_tractors_leased as tractors_leased,
            fs_trailers_owned as trailers_owned,
            fs_trailers_leased as trailers_leased,
            fs_haz_trailers_owned as haz_trailers_owned,
            fs_haz_trailers_leased as haz_trailers_leased,
            fs_haz_trucks_owned as haz_trucks_owned,
            fs_haz_trucks_leased as haz_trucks_leased,
            sum(fs_trucks_owned + fs_trucks_leased + fs_haz_trucks_owned + fs_haz_trucks_leased) as total_trucks,
            sum(fs_tractors_owned + fs_tractors_leased) as total_tractors,
            sum(fs_trailers_owned + fs_trailers_leased + fs_haz_trailers_owned + fs_haz_trailers_leased) as total_trailers,
            sum(fs_trucks_owned + fs_trucks_leased + fs_tractors_owned + fs_tractors_leased +fs_haz_trucks_owned + fs_haz_trucks_leased) as total_vehicles,
            sum(fs_trucks_owned + fs_tractors_owned + fs_haz_trucks_owned) as total_vehicles_owned,
            sum(fs_trucks_leased + fs_tractors_leased + fs_haz_trucks_leased) as total_vehicles_leased,			
            update_reason,
            fs_fleet_id,
            mcmis_hm_ind as hm_ind,
            mcmis_mcs150mileageyear as mcs150mileageyear,
            mcmis_mlg150 as mlg150,
            mcmis_tot_drs as tot_drs,
            mcmis_cdl_drs as cdl_drivers,
            mcmis_icc1 as mcn,
            fs_gvw_class_12 as gvw_class_12,
            fs_gvw_class_345 as gvw_class_345,
            fs_gvw_class_6 as gvw_class_6,
            fs_gvw_class_7 as gvw_class_7,
            fs_gvw_class_8 as gvw_class_8,
            fs_website as website,
            fs_sic as sic,
            fs_product_code as product_code
            

        from fleetseek.update_input
        where 
            update_reason like '%Fleet Discrepancy%' or
            update_reason like '%Trailer Discrepancy%'
        group by 
            fs_trucks_owned,
            fs_trucks_leased,
            fs_tractors_owned,
            fs_tractors_leased,
            fs_trailers_owned,
            fs_trailers_leased,
            fs_haz_trailers_owned,
            fs_haz_trailers_leased,
            fs_haz_trucks_owned,
            fs_haz_trucks_leased,
            update_reason,
            fs_fleet_id,
            mcmis_hm_ind,
            mcmis_mcs150mileageyear,
            mcmis_mlg150,
            mcmis_tot_drs,
            mcmis_cdl_drs,
            mcmis_icc1,
            fs_gvw_class_12,
            fs_gvw_class_345,
            fs_gvw_class_6,
            fs_gvw_class_7,
            fs_gvw_class_8,
            fs_sic,
            fs_product_code
            
    ) as sub_q
    where fleet.fleet_id = sub_q.fs_fleet_id
    returning
        fleet.total_vehicles,
        fleet.total_trailers,
        fleet.haz_trailers_owned,
        fleet.haz_trailers_leased,
        fleet.haz_trucks_owned,
        fleet.haz_trucks_leased,
        fleet.fleet_id
)
update fleetseek.fleet_primary
set
    total_vehicles = sub_a.total_vehicles,
    total_trailers = sub_a.total_trailers,
    hazmat_trucks = sub_a.hazmat_trucks,
    hazmat_trailers = sub_a.hazmat_trailers
from (
    select 
        total_vehicles,
        total_trailers,
        sum(haz_trucks_owned + haz_trucks_leased) as hazmat_trucks,
        sum(haz_trailers_owned + haz_trailers_leased) as hazmat_trailers,
        fleet_id
    from updated
    group by 
        total_vehicles,
        total_trailers,
        fleet_id
    ) as sub_a
where fleet_primary.fleet_id = sub_a.fleet_id

""")
conn.commit()

## Commodity update

In [ ]:
cur.execute("""
update fleetseek.fleet_primary
set
    commodity_id = sub_q.commodity_id,
    update_date = now()::date
from (
    select
        string_to_array(fs_commodities, ', ')::integer[] as commodity_id,
        fs_fleet_id
    from fleetseek.update_input
    where 
        update_reason like '%Commodity%' and 
        fs_commodities is not null
) as sub_q
where fleet_primary.fleet_id = sub_q.fs_fleet_id
""")
conn.commit()

## Trailer type updates

In [ ]:
cur.execute("""
update fleetseek.fleet_primary
set
    trailer_type_id = sub_q.trailer_type_id,
    update_date = now()::date
from (
    select 
        string_to_array(fs_trailer_types, ', ')::integer[] as trailer_type_id,
        fs_fleet_id
    from fleetseek.update_input
    where
        update_reason like '%Trailer Type%' and
        fs_trailer_types is not null
) as sub_q
where fleet_primary.fleet_id = sub_q.fs_fleet_id
""")
conn.commit()

## Changes to phone numbers

In [ ]:
# Updates regular phone numbers, fax numbers, and toll free numbers
cur.execute("""
update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    updated_date_time = now()
from (
    select
        fs_phone_number as phone_number,
        fs_phone_id
    from fleetseek.update_input
    where
        update_reason like '%Update Phone%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_phone_id;


update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    updated_date_time = now()
from (
    select
        fs_fax_number,
        fs_fax_id
    from fleetseek.update_input
    where
        update_reason like '%Update Fax%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_fax_id;


update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    updated_date_time = now()
from (
    select
        fs_tf_number,
        fs_toll_free_id
    from fleetseek.update_input
    where
        update_reason like '%Update TF%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_toll_free_id;

""")
conn.commit()

## Location changes

In [ ]:
# Specifically for physical address changes
# Going forward updates will exclusively use the address2 field
cur.execute("""

update fleetseek.location
set
    address1 = sub_q.address1,
    address2 = sub_q.address2,
    address3 = null,
    city = sub_q.city,
    county_fips = sub_q.county_fips,
    state_code = sub_q.state_code,
    zip_code = sub_q.zip_code,
    updated_date_time = now(),
    latitude = sub_q.latitude,
    longitude = sub_q.longitude,
    county_name = sub_q.county_name
from (
    select
        fs_phy_address1 as address1,
        fs_phy_address2 as address2,
        fs_phy_city as city,
        fs_phy_state_code as state_code,
        fs_phy_zip_code as zip_code,
        fs_phy_latitude as latitude,
        fs_phy_longitude as longitude,
        fs_phy_county_name as county_name,
        fs_physical_location_id as location_id
        fs_phy_county_fips as county_fips
    from fleetseek.update_input 
    where update_reason like '%Address Change%'
) as sub_q
where location.location_id = sub_q.location_id
""")
conn.commit()


In [ ]:
# Mailing address changes
# Going forward updates will exclusively use the address2 field
cur.execute("""
update fleetseek.location
set
    address1 = sub_q.address1,
    address2 = sub_q.address2,
    address3 = null,
    city = sub_q.city,
    county_fips = sub_q.county_fips,
    state_code = sub_q.state_code,
    zip_code = sub_q.zip_code,
    updated_date_time = now(),
    latitude = sub_q.latitude,
    longitude = sub_q.longitude,
    county_name = sub_q.county_name
from (
    select
        fs_mai_address1 as address1,
        fs_mai_address2 as address2,
        fs_mai_city as city,
        fs_mai_state_code as state_code,
        fs_mai_zip_code as zip_code,
        fs_mai_latitude as latitude,
        fs_mai_longitude as longitude,
        fs_mai_county_name as county_name,
        fs_mailing_location_id as location_id,
        fs_mai_county_fips as county_fips
    from fleetseek.update_input 
    where update_reason like '%Address Change%'
) as sub_q
where location.location_id = sub_q.location_id
""")
conn.commit()


## Company name changes

In [ ]:
cur.execute("""
update fleetseek.fleet
set 
    company_name = sub_q.company_name,
    company_sort_key = upper(replace(replace(replace(replace(sub_q.company_name, ' ', ''), '.', ''), '-', ''), '&', '')),
    update_date = now()::date
from (
    select 
        fs_company_name as company_name,
        fs_fleet_id
    from fleetseek.update_input
    where update_reason like '%Company Name Update%'
) as sub_q
where fleet.fleet_id = sub_q.fs_fleet_id

""")
conn.commit()

## Deactivations based on MCS-150 date or an inactive DOT

In [ ]:
cur.execute("""
update fleetseek.fleet
set 
    active_flag = 'N'
    update_date = now()::date
from (
    select 
        fs_active_flag,
        fs_fleet_id,
        update_reason
    from fleetseek.update_input
    where update_reason like '%Exceeds MCS%' or update_reason like '%Inactive DOT%'
) as sub_q
where fleet.fleet_id = sub_q.fleet_id
""")
conn.commit()

## Reactivations (Fleet table)

#### All subsequent changes will need to reflect in all relevant tables

In [ ]:
cur.execute("""
update fleetseek.fleet
set
    product_code = sub_q.product_code,
    company_name = sub_q.company_name,
    company_sort_key = sub_q.company_sort_key,
    website = sub_q.website,
    active_flag = 'Y',
    company_type_code = sub_q.company_type_code,
    fleet_type_id = sub_q.fleet_type_id,
    safety_rating_type_id = sub_q.safety_rating_type_id,
    trucks_owned = sub_q.trucks_owned,
    trucks_leased = sub_q.trucks_leased,
    tractors_owned = sub_q.tractors_owned,
    tractors_leased = sub_q.tractors_leased,
    trailers_owned = sub_q.trailers_owned,
    trailers_leased = sub_q.trailers_leased,
    gvw_class_12 = sub_q.gvw_class_12,
    gvw_class_345 = sub_q.gvw_class_345,
    gvw_class_6 = sub_q.gvw_class_6,
    gvw_class_7 = sub_q.gvw_class_7,
    gvw_class_8 = sub_q.gvw_class_8,
    sic_code = sub_q.sic_code,
    update_date = now()::date,
    haz_trailers_owned = sub_q.haz_trailers_owned,
    haz_trailers_leased = sub_q.haz_trailers_leased,
    haz_trucks_owned = sub_q.haz_trucks_owned,
    haz_trucks_leased = sub_q.haz_trucks_leased,
    vehicle_miles = sub_q.vehicle_miles,
    drivers = sub_q.drivers,
    cdl_drivers = sub_q.cdl_drivers,
    average_miles = sub_q.average_miles,
    year_mileage = sub_q.year_mileage,
    dot_hmind = sub_q.dot_hmind,
    total_trucks = sub_q.total_trucks,
    total_tractors = sub_q.total_tractors,
    total_trailers = sub_q.total_trailers,
    total_vehicles = sub_q.total_vehicles,
    total_vehicles_owned = sub_q.total_vehicles_owned,
    total_vehicles_leased = sub_q.total_vehicles_leased
from (
    select 
        initcap(coalesce(ui.mcmis_name_dba, ui.mcmis_name)) as company_name,
        upper(replace(replace(replace(replace(coalesce(ui.mcmis_name_dba, ui.mcmis_name), ' ', ''), '.', ''), '-', ''), '&', '')) as company_sort_key,
        ui.fs_website as website,
        sub_a."SIC" as sic_code,
        case
            when (substring(sub_a."SIC"::varchar, 3, 1) = '0') then right(sub_a."SIC"::varchar, 1)
            when (substring(sub_a."SIC"::varchar, 3, 1) <> '0') then right(sub_a."SIC"::varchar, 2)
        end as company_type_code,
        coalesce(sub_b.sub_b_product_code, sub_c.sub_c_product_code) as product_code,
        coalesce(sub_b.sub_b_fleet_type_id, sub_c.sub_c_fleet_type_id) as fleet_type_id,
        case 
            when (ui.mcmis_rating = 'C') then 107
            when (ui.mcmis_rating = 'S') then 109
            when (ui.mcmis_rating = 'U') then 110
        end as safety_rating_type_id,
        sum(ui.mcmis_owntruck + ui.fs_haz_trucks_owned) as trucks_owned,
        sum(ui.mcmis_trmtruck + ui.mcmis_trptruck + ui.fs_haz_trucks_leased) as trucks_leased,
        ui.mcmis_owntract as tractors_owned,
        sum(ui.mcmis_trmtract + ui.mcmis_trptract) as tractors_leased,
        sum(ui.mcmis_owntrail + ui.fs_haz_trailers_owned) as trailers_owned,
        sum(ui.mcmis_trmtrail + ui.mcmis_trptrail + ui.fs_haz_trailers_leased) as trailers_leased,
        ui.fs_gvw_class_12 as gvw_class_12,
        ui.fs_gvw_class_345 as gvw_class_345,
        ui.fs_gvw_class_6 as gvw_class_6,
        ui.fs_gvw_class_7 as gvw_class_7,
        ui.fs_gvw_class_8 as gvw_class_8,
        ui.fs_haz_trailers_owned as haz_trailers_owned,
        ui.fs_haz_trailers_leased as haz_trailers_leased,
        ui.fs_haz_trucks_owned as haz_trucks_owned,
        ui.fs_haz_trucks_leased as haz_trucks_leased,
        ui.mcmis_mlg150 as vehicle_miles,
        ui.mcmis_tot_drs as drivers,
        ui.mcmis_cdl_drs as cdl_drivers,
        ui.mcmis_mcs150mileageyear as year_mileage,
        ui.mcmis_hm_ind as dot_hmind,
        sum(ui.mcmis_owntruck + ui.mcmis_trmtruck + ui.mcmis_trptruck + ui.fs_haz_trucks_owned + ui.fs_haz_trucks_leased) as total_trucks,
        sum(ui.mcmis_owntract + ui.mcmis_trmtract + ui.mcmis_trptract) as total_tractors,
        sum(ui.mcmis_owntrail + ui.mcmis_trmtrail + ui.mcmis_trptrail + ui.fs_haz_trailers_owned + ui.fs_haz_trailers_leased) as total_trailers,
        sum(ui.mcmis_owntruck + ui.mcmis_trmtruck + ui.mcmis_trptruck + ui.fs_haz_trucks_owned + ui.fs_haz_trucks_leased + ui.mcmis_owntract + ui.mcmis_trmtract + ui.mcmis_trptract) as total_vehicles,
        sum(ui.mcmis_owntruck + ui.fs_haz_trucks_owned + ui.mcmis_owntract) as total_vehicles_owned,
        sum(ui.mcmis_trmtruck + ui.mcmis_trptruck + ui.fs_haz_trucks_leased + ui.mcmis_trmtract + ui.mcmis_trptract) as total_vehicles_leased,
        fs_fleet_id,
        ui.mcmis_average_miles as average_miles

        from fleetseek.update_input as ui
        left join (
            select tt.*, sub_ff.census_num, sub_ff."SIC"
            from (
                select 
                case 
                    when (sub_f.total_tractors = 0 and total_trailers > 0) then 'Utility'
                    when (sub_f.total_trailers = 1) then initcap("Trailer Type if 1 Trailer")
                    when (sub_f.total_trailers = 2) then initcap(coalesce("Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"))
                    when (sub_f.total_trailers = 3) then initcap(coalesce("Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"))
                    when (sub_f.total_trailers = 4) then initcap(coalesce("Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"))
                    when (sub_f.total_trailers = 5) then initcap(coalesce("Trailer Type if 5 or More trailers", "Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"))
                    when (sub_f.total_trailers = 6) then initcap(coalesce("Trailer type if 6 or more trailers", "Trailer Type if 5 or More trailers", "Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"))
                    when (sub_f.total_trailers = 7) then initcap(coalesce("Trailer type if 7 or more trailers", "Trailer type if 6 or more trailers", "Trailer Type if 5 or More trailers", "Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"))
                    when (sub_f.total_trailers > 7) then initcap(coalesce("Trailer type if 7 or more trailers", "Trailer type if 6 or more trailers", "Trailer Type if 5 or More trailers", "Trailer Type if 4 or more trailers", "Trailer Type if 3 Trailers", "Trailer Type if 2 Trailers", "Trailer Type if 1 Trailer"))

                end as trailer_type,
                sub_f.*
                from (
                    select 
                        sub_q.total_trailers, 
                        sub_q.census_num, 
                        sub_q.product_code, 
                        sub_q.total_tractors, 
                        sub_q.sub_q_commod_key, 
                        tk.*
                    from (
                        select concat( genfreight,  household,  usmail,  intermodal,  paperprod,  beverages,  coldfood,  produce,  meat, 
                                 livestock,  grainfeed,  farmsupp,  drybulk,  bldgmat,  metalsheet,  logpole,  construct,  machlrg,  oilfield,
                                 waterwell,  liqgas,  chem,  coalcoke,  garbage,  drivetow,  motorveh,  mobilehome,  utility,  cargoothr) as sub_q_commod_key,
                        *

                        from (
                            select
                                mcmis_census_num as census_num,
                                mcmis_class as class,
                                fs_product_code as product_code,

                                case 
                                    when(mcmis_genfreight is not null) then 'genfreight' else null
                                end as genfreight,
                                case
                                    when(mcmis_household is not null) then 'household' else null
                                end as household,
                                case
                                    when(mcmis_metalsheet is not null) then 'metalsheet' else null
                                end as metalsheet,
                                case
                                    when(mcmis_motorveh is not null) then 'motorveh' else null
                                end as motorveh,
                                case
                                    when(mcmis_drivetow is not null) then 'drivetow' else null
                                end as drivetow,
                                case
                                    when(mcmis_logpole is not null) then 'logpole' else null
                                end as logpole,
                                case
                                    when(mcmis_bldgmat is not null) then 'bldgmat' else null
                                end as bldgmat,
                                case
                                    when(mcmis_mobilehome is not null) then 'mobilehome' else null
                                end as mobilehome,
                                case
                                    when(mcmis_machlrg is not null) then 'machlrg' else null
                                end as machlrg,
                                case
                                    when(mcmis_produce is not null) then 'produce' else null
                                end as produce,
                                case
                                    when(mcmis_liqgas is not null) then 'liqgas' else null
                                end as liqgas,
                                case
                                    when(mcmis_intermodal is not null) then 'intermodal' else null
                                end as intermodal,
                                case
                                    when(mcmis_passengers is not null) then 'passengers' else null
                                end as passengers,
                                case
                                    when(mcmis_oilfield is not null) then 'oilfield' else null
                                end as oilfield,
                                case
                                    when(mcmis_livestock is not null) then 'livestock' else null
                                end as livestock,
                                case
                                    when(mcmis_grainfeed is not null) then 'grainfeed' else null
                                end as grainfeed,
                                case
                                    when(mcmis_coalcoke is not null) then 'coalcoke' else null
                                end as coalcoke,
                                case
                                    when(mcmis_meat is not null) then 'meat' else null
                                end as meat,
                                case
                                    when(mcmis_garbage is not null) then 'garbage' else null
                                end as garbage,
                                case
                                    when(mcmis_usmail is not null) then 'usmail' else null
                                end as usmail,
                                case
                                    when(mcmis_chem is not null) then 'chem' else null
                                end as chem,
                                case
                                    when(mcmis_drybulk is not null) then 'drybulk' else null
                                end as drybulk,
                                case
                                    when(mcmis_coldfood is not null) then 'coldfood' else null
                                end as coldfood,
                                case
                                    when(mcmis_beverages is not null) then 'beverages' else null
                                end as beverages,
                                case
                                    when(mcmis_paperprod is not null) then 'paperprod' else null
                                end as paperprod,
                                case
                                    when(mcmis_utility is not null) then 'utility' else null
                                end as utility,
                                case
                                    when(mcmis_farmsupp is not null) then 'farmsupp' else null
                                end as farmsupp,
                                case
                                    when(mcmis_construct is not null) then 'construct' else null
                                end as construct,
                                case
                                    when(mcmis_waterwell is not null) then 'waterwell' else null
                                end as waterwell,
                                case
                                    when(mcmis_cargoothr is not null) then 'cargoothr' else null
                                end as cargoothr,
                                sum(mcmis_owntrail + mcmis_trmtrail + mcmis_trptrail) as total_trailers,
                                sum(mcmis_owntract + mcmis_trmtract + mcmis_trptract) as total_tractors
                            from fleetseek.update_input
                            where 
                                mcmis_class in ('A', 'AB', 'ABD', 'ABG', 'ABL', 'AG', 'AGL', 'AL', 'B', 'BL') and
                                fs_product_code in ('OOD', 'NMC')

                            group by 

                                mcmis_genfreight, mcmis_household, mcmis_metalsheet, mcmis_motorveh, mcmis_drivetow, mcmis_logpole, mcmis_bldgmat, 
                                mcmis_mobilehome, mcmis_machlrg, mcmis_produce, mcmis_liqgas, mcmis_intermodal, mcmis_passengers, mcmis_oilfield, 
                                mcmis_livestock, mcmis_grainfeed, mcmis_coalcoke, mcmis_meat, mcmis_garbage, mcmis_usmail, mcmis_chem,
                                mcmis_drybulk, mcmis_coldfood, mcmis_beverages, mcmis_paperprod, mcmis_utility, mcmis_farmsupp, mcmis_construct,
                                mcmis_waterwell, mcmis_othercargo, mcmis_census_num, mcmis_cargoothr, mcmis_class, fs_product_code

                            having 

                                (sum(mcmis_owntruck + mcmis_trptruck+mcmis_trmtruck) >= 2 or
                                (sum(mcmis_owntruck + mcmis_trptruck + mcmis_trmtruck) = 1 and sum(mcmis_owntract + mcmis_trptract + mcmis_trmtract) = 0 and sum(mcmis_owntrail + mcmis_trmtrail + mcmis_trptrail) >= 2) or
                                (sum(mcmis_owntruck + mcmis_trptruck+mcmis_trmtruck) = 0 and sum(mcmis_owntract + mcmis_trptract + mcmis_trmtract) >= 1 and sum(mcmis_owntrail + mcmis_trmtrail + mcmis_trptrail) >= 2)) or
                                (sum(mcmis_owntract + mcmis_trmtract + mcmis_trptract) = 1 and sum(mcmis_owntrail + mcmis_trmtrail +mcmis_trptrail) <= 1 and sum(mcmis_owntruck + mcmis_trmtruck + mcmis_trptruck) = 0 and
                                mcmis_class similar to 'A%|B%' or mcmis_class = 'AB')

                        ) as sub_a
                    ) as sub_q
                left join 
                "Trailer_key" as tk
                on btrim(tk.commod_key) = btrim(sub_q.sub_q_commod_key)
                ) as sub_f
            ) as sub_ff
        join fleetseek.trailer_type as tt
        on sub_ff.trailer_type = tt.name
        ) as sub_a

        on ui.mcmis_census_num = sub_a.census_num


        left join (
            select 
                mcmis_census_num as sub_b_census_num,
                'OOD' as sub_b_product_code,
                105 as sub_b_fleet_type_id
            from fleetseek.update_input
            where 
                mcmis_passengers is null and
                mcmis_carship like 'C%' and
                (mcmis_class similar to 'A%|B%' or
                mcmis_class = 'AB') and 
                mcmis_mcs150_date > now() - interval '2 years' and
                mcmis_tot_buses = 0
            group by mcmis_census_num

            having 
                (sum(mcmis_owntract + mcmis_trmtract + mcmis_trptract) = 1 and sum(mcmis_owntrail + mcmis_trmtrail + mcmis_trptrail) <= 1) and 
                sum(mcmis_owntruck + mcmis_trmtruck + mcmis_trptruck) = 0
        ) as sub_b
        on ui.mcmis_census_num = sub_b.sub_b_census_num
        left join (
            select
                mcmis_census_num as sub_c_census_num,
                'NMC' as sub_c_product_code,
                104 as sub_c_fleet_type_id
            from fleetseek.update_input
            where
                mcmis_class in ('A', 'AB', 'ABD', 'ABG', 'ABL', 'AG', 'AGL', 'AL', 'B', 'BL')
            group by mcmis_census_num

            having 
                (sum(mcmis_owntruck + mcmis_trmtruck + mcmis_trptruck) >= 2 or
                (sum(mcmis_owntruck + mcmis_trmtruck + mcmis_trptruck) = 1 and sum(mcmis_owntract + mcmis_trmtract + mcmis_trptract) = 1) or
                (sum(mcmis_owntruck + mcmis_trmtruck + mcmis_trptruck) >= 1 and sum(mcmis_owntract + mcmis_trmtract + mcmis_trptract) = 0 and sum(mcmis_owntrail + mcmis_trmtrail + mcmis_trptrail) >= 2) or
                (sum(mcmis_owntruck + mcmis_trmtruck + mcmis_trptruck) = 0 and sum(mcmis_owntract + mcmis_trmtract + mcmis_trptract) >= 1 and sum(mcmis_owntrail + mcmis_trmtrail + mcmis_trptrail) >= 2))
        ) as sub_c
        on ui.mcmis_census_num = sub_c_census_num
        group by 
            ui.mcmis_name,
            ui.mcmis_name_dba,
            ui.fs_website,
            sub_a."SIC",
            sub_b.sub_b_product_code,
            sub_b.sub_b_fleet_type_id,
            sub_c.sub_c_product_code,
            sub_c.sub_c_fleet_type_id,
            ui.mcmis_rating,
            ui.mcmis_owntract,
            ui.fs_gvw_class_12,
            ui.fs_gvw_class_345,
            ui.fs_gvw_class_6,
            ui.fs_gvw_class_7,
            ui.fs_gvw_class_8,
            ui.fs_haz_trailers_owned,
            ui.fs_haz_trailers_leased,
            ui.fs_haz_trucks_owned,
            ui.fs_haz_trucks_leased ,
            ui.mcmis_mlg150,
            ui.mcmis_tot_drs,
            ui.mcmis_cdl_drs,
            ui.mcmis_mcs150mileageyear,
            ui.mcmis_hm_ind,
            ui.fs_fleet_id,
            ui.mcmis_average_miles

) as sub_q
where fleet.fleet_id = sub_q.fs_fleet_id

""")
conn.commit()

## Phone number

In [ ]:
# Updates that exist

cur.execute("""
-- regular phone numbers
update fleetseek.phone_number
set 
    phone_number = sub_q.phone_number,
    updated_date_time = now()::timestamp without time zone
from (
    select
        fs_phone_number,
        fs_phone_id
    from fleetseek.update_input
    where update_reason like '%Update Phone%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_phone_id;

-- fax numbers
update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    updated_date_time = now()::timestamp without time zone
from (
    select 
        fs_fax_number,
        fs_fax_fax_id
    from fleetseek.update_input
    where update_reason like '%Fax%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_fax_id;


-- toll free
update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    updated_date_time = now()::timestamp without time zone
from (
    select
        fs_tf_number,
        fs_toll_free_id
    from fleetseek.update_input
    where update_reason like '%TF%'
) as sub_q
where phone_number.phone_number_id = sub_q.fs_toll_free_id;
""")
conn.commit()

## Company name change

In [ ]:
cur.execute("""
update fleetseek.fleet
set 
    company_name = sub_q.company_name,
    company_sort_key = sub_q.company_sort_key,
    update_date = now()::date
from (
    select 
        fs_company_name as company_name,
        upper(replace(replace(replace(replace(ui.fs_company_name, ' ', ''), '.', ''), '-', ''), '&', '')) as company_sort_key,
        fs_fleet_id
    from fleetseek.update_input as ui
    where update_reason like '%Company Name%'
) as sub_q
where fleet.fleet_id = sub_q.fs_fleet_id
""")
conn.commit()

# These next cells are contact specific updates

## Contact Name and job info

In [ ]:
cur.execute("""
update fleetseek.contact
set 
    honorific = sub_q.honorific,
    first_name = sub_q.first_name,
    last_name = sub_q.last_name,
    middle_name = sub_q.middle_name,
    suffix = sub_q.suffix,
    updated_date_time = now()::date,
    original_job_title = sub_q.original_job_title,
    active = sub_q.active_contact,
    job_title_id = sub_q.job_title_id
from (
    select 
        honorific,
        first_name,
        last_name,
        middle_name,
        suffix,
        original_job_title,
        active_contact,
        job_title_id,
        contact_id as update_contact_id
    from fleetseek.contact_update_input
    where note like '%Update Contact Name%'
) as sub_q
where contact.contact_id = sub_q.update_contact_id
""")
conn.commit()

## Contact email

In [ ]:
cur.execute("""
update fleetseek.email
set
    email_address = sub_q.email_address
from (
    select 
        email_address,
        email_id
    from fleetseek.contact_update_input
    where note like '%Update Email%'
) as sub_q
where email.email_id = sub_q.email_id
"""
)
conn.commit()

## Contact phone info

In [ ]:
cur.execute("""
update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    phone_number_extension = sub_q.phone_number_extension,
    updated_date_time = now()

from (
    select 
        phone_number,
        phone_number_extension,
        phone_number_id
    from fleetseek.contact_update_input
    where note like '%Update Phone%'
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;

update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    phone_number_extension = sub_q.phone_number_extension,
    updated_date_time = now()

from (
    select 
        phone_number,
        phone_number_extension,
        phone_number_id
    from fleetseek.contact_update_input
    where note like '%Update Fax%'
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;

update fleetseek.phone_number
set
    phone_number = sub_q.phone_number,
    phone_number_extension = sub_q.phone_number_extension,
    updated_date_time = now()

from (
    select 
        phone_number,
        phone_number_extension,
        phone_number_id
    from fleetseek.contact_update_input
    where note like '%Update TF%'
) as sub_q
where phone_number.phone_number_id = sub_q.phone_number_id;

""")
conn.commit()

## Contact address

In [ ]:
cur.execute("""
update fleetseek.location
set
    address1 = sub_q.address1,
    address2 = sub_q.address2,
    address3 = sub_q.address3,
    city = sub_q.city,
    county_fips = sub_q.county_fips,
    state_code = sub_q.state_code,
    zip_code = sub_q.zip_code,
    country_code = sub_q.country_code,
    updated_date_time = now(),
    active = sub_q.active,
    county_name = sub_q.county_name
from (
    select 
        phy_address as address1,
        phy_address2 as address2,
        phy_address3 as address3,
        phy_city as city,
        phy_countyfips as county_fips,
        phy_state_code as state_code,
        phy_zip_code as zip_code,
        phy_country_code as country_code,
        phy_active as active,
        phy_county_name as county_name,
        phy_location_id
    from fleetseek.manual_review_contact_input
    where note like '%Update Physical Address%'

) as sub_q
where location.location_id = sub_q.phy_location_id;


update fleetseek.location
set
    address1 = sub_q.address1,
    address2 = sub_q.address2,
    address3 = sub_q.address3,
    city = sub_q.city,
    county_fips = sub_q.county_fips,
    state_code = sub_q.state_code,
    zip_code = sub_q.zip_code,
    country_code = sub_q.country_code,
    updated_date_time = now(),
    active = sub_q.active,
    county_name = sub_q.county_name
from (
    select 
        mai_address as address1,
        mai_address2 as address2,
        mai_address3 as address3,
        mai_city as city,
        mai_countyfips as county_fips,
        mai_state_code as state_code,
        mai_zip_code as zip_code,
        mai_country_code as country_code,
        mai_active as active,
        mai_county_name as county_name,
        mai_location_id
    from fleetseek.manual_review_contact_input
    where note like '%Update Mailing Address%'

) as sub_q
where location.location_id = sub_q.mai_location_id;
""")
conn.commit()